In [1]:
import pandas as pd
import json
import gzip
import os
import numpy as np
import sys
from tqdm import tqdm

# Gathering all of the data

## Mapping DF

In [2]:
df = pd.read_json('files-cases.json')
df.head()

,file_name,data_format,access,data_category,file_size,cases,annotations
0,2b5c518f-8327-478a-a282-01bfe59aca4c.htseq.cou...,TXT,open,Transcriptome Profiling,257725,"[{'project': {'project_id': 'TCGA-BRCA'}, 'cas...",NaN
1,64262505-9f17-4989-94c9-fc6db56ca676.htseq.cou...,TXT,open,Transcriptome Profiling,256496,"[{'project': {'project_id': 'TCGA-BRCA'}, 'cas...",NaN
2,f5d3e683-9177-45fc-93e3-357bf7366ac4.htseq.cou...,TXT,open,Transcriptome Profiling,252976,"[{'project': {'project_id': 'TCGA-BRCA'}, 'cas...",NaN
3,6999d309-8502-49ee-8d80-9d0bde00081f.htseq.cou...,TXT,open,Transcriptome Profiling,250749,"[{'project': {'project_id': 'TCGA-BRCA'}, 'cas...",NaN
4,aa623193-428e-41b8-b051-2d9693d852f8.htseq.cou...,TXT,open,Transcriptome Profiling,259222,"[{'project': {'project_id': 'TCGA-BRCA'}, 'cas...",NaN


In [3]:
def extract_case_id(row):
    cases = row['cases']
    return cases[0]['case_id']
    

df['case_id'] = df.apply(extract_case_id, axis=1)

In [4]:
df.head()

,file_name,data_format,access,data_category,file_size,cases,annotations,case_id
0,2b5c518f-8327-478a-a282-01bfe59aca4c.htseq.cou...,TXT,open,Transcriptome Profiling,257725,"[{'project': {'project_id': 'TCGA-BRCA'}, 'cas...",NaN,4d0fec97-e024-4608-a0cc-426a3decc7b1
1,64262505-9f17-4989-94c9-fc6db56ca676.htseq.cou...,TXT,open,Transcriptome Profiling,256496,"[{'project': {'project_id': 'TCGA-BRCA'}, 'cas...",NaN,bb8d42d3-ad65-4d88-ae1d-f9aadfc7962d
2,f5d3e683-9177-45fc-93e3-357bf7366ac4.htseq.cou...,TXT,open,Transcriptome Profiling,252976,"[{'project': {'project_id': 'TCGA-BRCA'}, 'cas...",NaN,c694615c-b1c6-499c-8058-995633ebf948
3,6999d309-8502-49ee-8d80-9d0bde00081f.htseq.cou...,TXT,open,Transcriptome Profiling,250749,"[{'project': {'project_id': 'TCGA-BRCA'}, 'cas...",NaN,1285eb55-415c-494a-aa58-936f0427cdd0
4,aa623193-428e-41b8-b051-2d9693d852f8.htseq.cou...,TXT,open,Transcriptome Profiling,259222,"[{'project': {'project_id': 'TCGA-BRCA'}, 'cas...",NaN,6e126b73-d3e8-4641-a128-306f3b313e40


In [5]:
mapping_df = df[['file_name', 'case_id']]

In [6]:
mapping_df.head()

,file_name,case_id
0,2b5c518f-8327-478a-a282-01bfe59aca4c.htseq.cou...,4d0fec97-e024-4608-a0cc-426a3decc7b1
1,64262505-9f17-4989-94c9-fc6db56ca676.htseq.cou...,bb8d42d3-ad65-4d88-ae1d-f9aadfc7962d
2,f5d3e683-9177-45fc-93e3-357bf7366ac4.htseq.cou...,c694615c-b1c6-499c-8058-995633ebf948
3,6999d309-8502-49ee-8d80-9d0bde00081f.htseq.cou...,1285eb55-415c-494a-aa58-936f0427cdd0
4,aa623193-428e-41b8-b051-2d9693d852f8.htseq.cou...,6e126b73-d3e8-4641-a128-306f3b313e40


In [7]:
mapping_df.shape

(1164, 2)

In [8]:
len(set(mapping_df['case_id']))

1036

Although it has 1164 lines, it only says about 1036 patients, because some patients have multiple files

In [9]:
mapping_df.isnull().sum()

file_name    0
case_id      0
dtype: int64

## Clinical DF

In [10]:
clinical = pd.read_csv('Clinical_Clean_Doria.csv')
clinical.head()

,Unnamed: 0,case_id,submitter_id,tumor_stage,age_at_diagnosis,prior_malignancy,race_asian,race_black or african american,race_white
0,0,dd96a9c7-899c-47cd-a0f9-b149ed07a5d6,TCGA-D8-A1JB,stage iib,54.306849,0,0,0,1
1,1,3f834fa7-6d7b-4b85-98c0-5c55d55b6c95,TCGA-B6-A0IE,stage iiia,38.306849,0,0,1,0
2,2,451e1a67-47e6-4738-99d7-fb7771ef61a3,TCGA-B6-A0RP,not reported,73.810959,0,0,0,1
3,3,178b2c48-c07d-422e-ae17-8bcfd996ad51,TCGA-B6-A0X1,not reported,48.284932,0,0,0,1
4,4,dddd8e2f-e540-418a-b02e-698d18a12c14,TCGA-A7-A26H,stage iia,72.391781,1,0,0,1


In [11]:
clinical = clinical.drop(columns=['Unnamed: 0'])

In [12]:
clinical[clinical.isnull()].sum()

case_id                           0.0
submitter_id                      0.0
tumor_stage                       0.0
age_at_diagnosis                  0.0
prior_malignancy                  0.0
race_asian                        0.0
race_black or african american    0.0
race_white                        0.0
dtype: float64

In [13]:
print(mapping_df.shape)
print(clinical.shape)

(1164, 2)
(1036, 8)


In [14]:
joined = pd.merge(mapping_df, clinical, how='inner', on="case_id")
joined.head()

,file_name,case_id,submitter_id,tumor_stage,age_at_diagnosis,prior_malignancy,race_asian,race_black or african american,race_white
0,2b5c518f-8327-478a-a282-01bfe59aca4c.htseq.cou...,4d0fec97-e024-4608-a0cc-426a3decc7b1,TCGA-AR-A252,stage i,50.989041,0,0,0,1
1,64262505-9f17-4989-94c9-fc6db56ca676.htseq.cou...,bb8d42d3-ad65-4d88-ae1d-f9aadfc7962d,TCGA-AO-A1KS,stage iia,69.123288,0,0,0,1
2,f5d3e683-9177-45fc-93e3-357bf7366ac4.htseq.cou...,c694615c-b1c6-499c-8058-995633ebf948,TCGA-E9-A1RF,stage iiia,68.819178,0,0,0,1
3,94bedc10-62c2-4bd2-bad9-0ec08c9bf5e9.htseq.cou...,c694615c-b1c6-499c-8058-995633ebf948,TCGA-E9-A1RF,stage iiia,68.819178,0,0,0,1
4,6999d309-8502-49ee-8d80-9d0bde00081f.htseq.cou...,1285eb55-415c-494a-aa58-936f0427cdd0,TCGA-V7-A7HQ,stage iiia,75.846575,0,0,1,0


In [15]:
joined.shape

(1164, 9)

Perfect, we got all the files_names, so we can add the genes.

## Getting the genes

First, let's look at an example file:

In [16]:
test = pd.read_csv('Genes_Files/0a2c1866-d6e1-4add-bcb0-b297ba1394ac.txt', sep='\t', header=None, names=['gene', 'count'])
test['count'] = test['count'].astype('float')
test.head()

,gene,count
0,ENSG00000000003.13,2569.0
1,ENSG00000000005.5,1.0
2,ENSG00000000419.11,3180.0
3,ENSG00000000457.12,3332.0
4,ENSG00000000460.15,1621.0


In [17]:
genes = pd.pivot_table(test, values='count', columns=['gene'])
genes.head()

gene,ENSG00000000003.13,ENSG00000000005.5,ENSG00000000419.11,ENSG00000000457.12,ENSG00000000460.15,ENSG00000000938.11,ENSG00000000971.14,ENSG00000001036.12,ENSG00000001084.9,ENSG00000001167.13,...,ENSGR0000275287.3,ENSGR0000276543.3,ENSGR0000277120.3,ENSGR0000280767.1,ENSGR0000281849.1,__alignment_not_unique,__ambiguous,__no_feature,__not_aligned,__too_low_aQual
count,2569.0,1.0,3180.0,3332.0,1621.0,530.0,7282.0,3312.0,2642.0,3322.0,...,0.0,0.0,0.0,0.0,0.0,23748640.0,3368739.0,3069305.0,0.0,0.0


In [18]:
genes_dataframe = pd.DataFrame(columns=genes.columns)
root = 'Genes_Files/'
for row in tqdm(joined.iterrows()):
    row = row[1]
    name = row['file_name']
    name = root + name[:name.find('.')] + '.txt'
    features = pd.read_csv(name, sep='\t', header=None, names=['gene', 'count']) 
    features['count'] = features['count'].astype('float')
    pivot_df = pd.pivot_table(features, values='count', columns=['gene'])
    genes_dataframe = pd.concat([genes_dataframe, pivot_df], ignore_index=True)
genes_dataframe['file_name'] = joined['file_name']

1164it [08:17,  1.37it/s]


In [19]:
genes_dataframe.head()

gene,ENSG00000000003.13,ENSG00000000005.5,ENSG00000000419.11,ENSG00000000457.12,ENSG00000000460.15,ENSG00000000938.11,ENSG00000000971.14,ENSG00000001036.12,ENSG00000001084.9,ENSG00000001167.13,...,ENSGR0000276543.3,ENSGR0000277120.3,ENSGR0000280767.1,ENSGR0000281849.1,__alignment_not_unique,__ambiguous,__no_feature,__not_aligned,__too_low_aQual,file_name
0,2608.0,289.0,1661.0,2081.0,684.0,1261.0,8544.0,4815.0,2879.0,4088.0,...,0.0,0.0,0.0,0.0,33323894.0,3232629.0,5808920.0,0.0,0.0,2b5c518f-8327-478a-a282-01bfe59aca4c.htseq.cou...
1,5789.0,34.0,2538.0,2572.0,1292.0,284.0,2934.0,5797.0,3401.0,5931.0,...,0.0,0.0,0.0,0.0,19183901.0,2673271.0,6146344.0,0.0,0.0,64262505-9f17-4989-94c9-fc6db56ca676.htseq.cou...
2,4544.0,1881.0,1565.0,1356.0,294.0,1006.0,24121.0,3695.0,5097.0,2025.0,...,0.0,0.0,0.0,0.0,23874394.0,2400693.0,5278313.0,0.0,0.0,f5d3e683-9177-45fc-93e3-357bf7366ac4.htseq.cou...
3,676.0,41.0,3732.0,3155.0,1010.0,910.0,5382.0,3559.0,2968.0,3704.0,...,0.0,0.0,0.0,0.0,21737695.0,3123055.0,5182286.0,0.0,0.0,94bedc10-62c2-4bd2-bad9-0ec08c9bf5e9.htseq.cou...
4,2299.0,5.0,708.0,388.0,93.0,113.0,674.0,2219.0,871.0,923.0,...,0.0,0.0,0.0,0.0,20631886.0,2996831.0,2104393.0,0.0,0.0,6999d309-8502-49ee-8d80-9d0bde00081f.htseq.cou...


In [20]:
genes_dataframe.shape

(1164, 60489)

## Putting everything together

In [21]:
joined = pd.merge(joined, genes_dataframe, how='inner', on="file_name")
joined.head()

,file_name,case_id,submitter_id,tumor_stage,age_at_diagnosis,prior_malignancy,race_asian,race_black or african american,race_white,ENSG00000000003.13,...,ENSGR0000275287.3,ENSGR0000276543.3,ENSGR0000277120.3,ENSGR0000280767.1,ENSGR0000281849.1,__alignment_not_unique,__ambiguous,__no_feature,__not_aligned,__too_low_aQual
0,2b5c518f-8327-478a-a282-01bfe59aca4c.htseq.cou...,4d0fec97-e024-4608-a0cc-426a3decc7b1,TCGA-AR-A252,stage i,50.989041,0,0,0,1,2608.0,...,0.0,0.0,0.0,0.0,0.0,33323894.0,3232629.0,5808920.0,0.0,0.0
1,64262505-9f17-4989-94c9-fc6db56ca676.htseq.cou...,bb8d42d3-ad65-4d88-ae1d-f9aadfc7962d,TCGA-AO-A1KS,stage iia,69.123288,0,0,0,1,5789.0,...,0.0,0.0,0.0,0.0,0.0,19183901.0,2673271.0,6146344.0,0.0,0.0
2,f5d3e683-9177-45fc-93e3-357bf7366ac4.htseq.cou...,c694615c-b1c6-499c-8058-995633ebf948,TCGA-E9-A1RF,stage iiia,68.819178,0,0,0,1,4544.0,...,0.0,0.0,0.0,0.0,0.0,23874394.0,2400693.0,5278313.0,0.0,0.0
3,94bedc10-62c2-4bd2-bad9-0ec08c9bf5e9.htseq.cou...,c694615c-b1c6-499c-8058-995633ebf948,TCGA-E9-A1RF,stage iiia,68.819178,0,0,0,1,676.0,...,0.0,0.0,0.0,0.0,0.0,21737695.0,3123055.0,5182286.0,0.0,0.0
4,6999d309-8502-49ee-8d80-9d0bde00081f.htseq.cou...,1285eb55-415c-494a-aa58-936f0427cdd0,TCGA-V7-A7HQ,stage iiia,75.846575,0,0,1,0,2299.0,...,0.0,0.0,0.0,0.0,0.0,20631886.0,2996831.0,2104393.0,0.0,0.0


In [22]:
joined.shape

(1164, 60497)

## Exporting the data

Before exporting, I will remove unnecessary columns, to make the file a bit lighter:

In [23]:
joined = joined.drop(columns=['file_name', 'case_id', 'submitter_id', '__alignment_not_unique', '__ambiguous', '__no_feature', '__not_aligned', '__too_low_aQual'])

In [24]:
joined.to_csv("Joined_Data_All_Genes.csv")